In [1]:
%%capture
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import shutil
from glob import glob
import os

iris = datasets.load_iris()
iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(iris_train, iris.target)

# Infer the signature from the training dataset and model's predictions
signature = infer_signature(iris_train, clf.predict(iris_train))

# Conda environment
custom_env = _mlflow_conda_env(
    additional_conda_deps=None,
    additional_pip_deps=["xgboost==1.5.2"],
    additional_conda_channels=None,
)

# Sample
input_sample = iris_train.sample(n=1)
output_sample = clf.predict(input_sample)

# Delete previous experiments by erase the folser mlruns
if os.path.isdir("mlruns"):
    shutil.rmtree("./mlruns")

# Log the scikit-learn model with the custom signature
mlflow.sklearn.log_model(
    clf,
    artifact_path="iris_rf",
    conda_env=custom_env,
    signature=signature,
    input_example=input_sample,
)

In [2]:
destination_path = "models"
# Move model.pkl using shutil
for file in glob("mlruns/*/*/*/*/*"):
    if file.endswith("model.pkl"):
        print(file)  # Output: model.pkl
        if os.path.isfile(file):
            os.remove(destination_path + "/model.pkl")
            shutil.move(file, destination_path)
        else:
            shutil.move(file, destination_path)
    elif file.endswith("input_example.json"):
        print(file)  # Output: input_example.json
        if os.path.isfile(file):
            os.remove(destination_path + "/input_example.json")
            shutil.move(file, destination_path)
        else:
            shutil.move(file, destination_path)
    elif file.endswith("conda.yaml"):
        print(file)  # Output: conda.yaml
        if os.path.isfile(file):
            os.remove(destination_path + "/conda.yaml")
            shutil.move(file, destination_path)
        else:
            shutil.move(file, destination_path)

mlruns\0\dea885da1de943f88583fb951d7e7232\artifacts\iris_rf\conda.yaml
mlruns\0\dea885da1de943f88583fb951d7e7232\artifacts\iris_rf\input_example.json
mlruns\0\dea885da1de943f88583fb951d7e7232\artifacts\iris_rf\model.pkl


In [3]:
%%writefile models/score.py
import joblib
import json
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "models/model.pkl")
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)


@input_schema("data", NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # You can return any JSON-serializable object.
        return "Here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

Overwriting models/score.py
